In [9]:
import warnings
warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from prepare import get_311_data

spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

In [3]:
# reading in data and saving to separate DFs
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

# converting case_closed and case_late from strings to booleans
# this will allow us to perform boolean operations on these columns should we need to in the future
case = case.withColumn("case_closed", expr('case_closed == "YES"'))\
.withColumn("case_late", expr('case_late == "YES"'))

# converting council_district to string since we won't be performing mathematical operations on it
case = case.withColumn("council_district", col("council_district").cast("string"))

# setting date and time format
fmt = "M/d/yy H:mm"

# converting dates to date time format
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

### How many different cases are there, by department?

In [8]:
(
    case.groupBy("department")
    .count()
)

AnalysisException: cannot resolve '`department`' given input columns: [SLA_days, SLA_due_date, case_closed, case_closed_date, case_due_date, case_id, case_late, case_opened_date, case_status, council_district, dept_division, num_days_late, request_address, service_request_type, source_id];;
'Aggregate ['department], ['department, count(1) AS count#195L]
+- Project [case_id#36, case_opened_date#133, case_closed_date#148, SLA_due_date#39, case_late#103, num_days_late#41, case_closed#88, dept_division#43, service_request_type#44, SLA_days#45, case_status#46, source_id#47, request_address#48, council_district#118, to_timestamp('case_opened_date, Some(M/d/yy H:mm)) AS case_due_date#163]
   +- Project [case_id#36, case_opened_date#133, to_timestamp('case_opened_date, Some(M/d/yy H:mm)) AS case_closed_date#148, SLA_due_date#39, case_late#103, num_days_late#41, case_closed#88, dept_division#43, service_request_type#44, SLA_days#45, case_status#46, source_id#47, request_address#48, council_dist

In [ ]:
    # then pass to pandas
    .toPandas()
    .set_index("department")["count"]
    .sort_values(ascending=False)
    .plot.barh(width=1, edgecolor="black", figsize=(14, 5))
)

plt.title("Number of Cases By Department")
plt.ylabel("")
plt.xlabel("Number of Cases")

### Does the percentage of cases that are late vary by department?

In [10]:
get_311_data()

(DataFrame[source_id: string, source_username: string],
 DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int],
 DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string])

### On average, how late are the late cases by department?

### What is the service type that is the most late? Just for Parks & Rec?

### For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

### Does whether or not its a weekend matter for when a case is opened/closed?

### On average, how many cases are opened a day for the Customer Service department?

### Does the number of service requests for the solid waste department vary by day of the week?